In [1]:
suppressMessages(suppressWarnings(library(GenomicDataCommons)))
suppressMessages(suppressWarnings(library(data.table)))
library(dplyr)
library(DESeq2)
library(biomaRt)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:GenomicDataCommons’:

    count, filter, select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The follow

In [2]:
manifest <- fread("gdc_manifest_20210811_142726.txt", header = T)

In [3]:
manifest

id,filename,md5,size,state
<chr>,<chr>,<chr>,<int>,<chr>
40832679-44df-447a-8096-7f806b08e35e,7ef16280-e921-4585-bca7-8429e6089a34.htseq.counts.gz,288669d08e12ff7c9fc5ff6d7da0cd39,250070,validated
5884e418-e811-4850-bcb3-89e09572de29,677291bf-6149-4c97-b958-dd2b0219bfe5.htseq.counts.gz,d8d1d11006853e9b9e93b06b14ea2177,248661,validated
6986b7dc-25ed-4ae9-81b4-0892474d9e67,fc6f1676-b097-40f8-9c0c-fc8142cf5be3.htseq.counts.gz,fe3aa6012c5ba299453a8fa767af570d,250712,validated
b8da3f94-6213-4636-a8f0-26a4b419e3fb,9c77be54-d58e-4dec-8894-acb71a12cac5.htseq.counts.gz,68a22fe68c82d3925fe6e260aa1f9a44,245962,validated


In [4]:
for(i in 1:nrow(manifest)) {
    # download the donors bam file
    command= paste("module load gdc-client && gdc-client download ", manifest$id[i],
                    "--no-annotations",
                    "--no-related-files", sep = " ")
    print(command)
    try(system(command))
}

[1] "module load gdc-client && gdc-client download  40832679-44df-447a-8096-7f806b08e35e --no-annotations --no-related-files"
[1] "module load gdc-client && gdc-client download  5884e418-e811-4850-bcb3-89e09572de29 --no-annotations --no-related-files"
[1] "module load gdc-client && gdc-client download  6986b7dc-25ed-4ae9-81b4-0892474d9e67 --no-annotations --no-related-files"
[1] "module load gdc-client && gdc-client download  b8da3f94-6213-4636-a8f0-26a4b419e3fb --no-annotations --no-related-files"


In [5]:
### files to be downloaded

# read in the .txt file for tissue type as well as the larger gene expression file\
ptm <- proc.time()
sample_attributes <- fread(file = "/data/timonaj/gene_variability/GTEx_v7_Annotations_SampleAttributesDS.txt")
gene_tpm <- fread(file = "/data/timonaj/gene_variability/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct")
gene_tpm_copy <- as.data.frame(gene_tpm)
print("download completed in")
print(proc.time() - ptm)

[1] "download completed in"
   user  system elapsed 
 38.757  10.850  57.962 


In [6]:
gene_counts <- fread(file = "./GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct.gz")

Warning message in fread(file = "./GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct.gz"):
“Discarded single-line footer: <<"><meta name=gtex-build-date content="8/3/2021, 3:24:42 PM"><meta name=viewport content="width=device-width,initial-scale=1"><meta name=keywords content="GTEx, genotype tissue expression, eQTL, transcriptome, gene expression, human, multi-tissue, human transcriptome, human genome, quantitative trait loci, genetic association, isoform expression, gene eQTL, cis-eQTL, trans-eQTL, splice-QTL, splicing, protein truncating variant, genotype, expression, tissue, biobank, biobanking, tissue samples, no>>”


In [7]:
head(gene_counts)

<!DOCTYPE,html><html,lang=en><head><title>GTEx,Portal</title><meta,charset=utf-8><meta,name=gtex-version,"content=""cl292"
<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>


In [8]:
# log + 1 every value
tpm_length <- 3:length(gene_tpm_copy)
sample_tpm <- head(gene_tpm_copy)
gene_tpm_copy[,tpm_length] <- gene_tpm_copy[,tpm_length] + 1
gene_tpm_copy[,tpm_length] <- log(gene_tpm_copy[,tpm_length])

In [9]:
# convert the column names into the tissue type so it's easier to subset
ptm <- proc.time()
convoluted_tissue_types <- colnames(gene_tpm_copy)[tpm_length]
new_tt_colnames <- vector(mode = "numeric", length(convoluted_tissue_types))
for (i in 1:length(convoluted_tissue_types)) {
  new_tt_colnames[i] <- sample_attributes[sample_attributes$SAMPID == convoluted_tissue_types[i]]$SMTS
}
names(gene_tpm_copy) <- c(colnames(gene_tpm_copy)[1:2], new_tt_colnames)
print("rownames computed in")
print(proc.time() - ptm)

[1] "rownames computed in"
   user  system elapsed 
  5.445   0.307   6.163 


In [10]:
skin_samples<- fread("skin_counts.txt.gz") %>% as.data.frame()
row.names(skin_samples) <- skin_samples$V1
skin_samples <- skin_samples[,-1]

Warning message in fread("skin_counts.txt.gz"):
“Detected 1203 column names but the data has 1204 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [11]:
total_samples <- skin_samples
for(i in 1:nrow(manifest)) {
    cur_file <- fread(paste("./",manifest$id[i],"/",manifest$filename[i], sep=""))  %>% as.data.frame()
    row.names(cur_file) <- cur_file$V1
    colnames(cur_file) <- c("names", manifest$filename[i])
    total_samples <- merge(total_samples, cur_file, by=0)
    row.names(total_samples) <- total_samples$Row.names
    total_samples <- total_samples[,-c(1, (length(total_samples)-1))]
}

In [12]:
dim(total_samples)
head(total_samples)

[1] 31919  1207

,Skin,Skin.1,Skin.2,Skin.3,Skin.4,Skin.5,Skin.6,Skin.7,Skin.8,Skin.9,⋯,Skin.1197,Skin.1198,Skin.1199,Skin.1200,Skin.1201,Skin.1202,7ef16280-e921-4585-bca7-8429e6089a34.htseq.counts.gz,677291bf-6149-4c97-b958-dd2b0219bfe5.htseq.counts.gz,fc6f1676-b097-40f8-9c0c-fc8142cf5be3.htseq.counts.gz,9c77be54-d58e-4dec-8894-acb71a12cac5.htseq.counts.gz
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000000005.5,0,4,0,14,0,4,0,0,0,0,⋯,69,32,99,210,41,20,0,2,0,0
ENSG00000001561.6,142,173,152,529,265,68,682,253,122,192,⋯,322,173,769,373,484,1169,153,598,30,4
ENSG00000004799.7,127,730,93,459,213,361,163,209,161,374,⋯,15267,657,17532,27517,27500,44762,4909,304,339,119
ENSG00000004848.6,0,0,2,3,0,0,1,0,0,0,⋯,2,0,2,2,14,2,57,8,7,0
ENSG00000005022.5,14187,10090,12616,9202,6329,9507,12311,6041,7872,15840,⋯,7403,20011,9841,9442,16210,13354,21438,7200,10217,30979
ENSG00000005073.5,507,290,506,979,986,445,869,553,563,594,⋯,109,78,108,72,70,396,455,153,1054,195


In [111]:
countData <- total_samples
condition <- factor(c(rep("A", (length(total_samples) - 4)), rep("B",4)))
dds <- DESeqDataSetFromMatrix(countData, DataFrame(condition), ~ condition)

converting counts to integer mode



In [119]:
dds

class: DESeqDataSet 
dim: 31919 1207 
metadata(1): version
assays(1): counts
rownames(31919): ENSG00000000005.5 ENSG00000001561.6 ...
  ENSG00000273489.1 ENSG00000273493.1
rowData names(0):
colnames(1207): Skin Skin.1 ...
  fc6f1676-b097-40f8-9c0c-fc8142cf5be3.htseq.counts.gz
  9c77be54-d58e-4dec-8894-acb71a12cac5.htseq.counts.gz
colData names(1): condition

In [121]:
dds <- DESeq(dds)
dds <- estimateSizeFactors(dds)
ddsCounts <- counts(dds, normalized=TRUE)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 537 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [127]:
logfcvals<-results(dds)

In [153]:
write.table(logfcvals, file = "./skin_logfc.txt", quote = F, sep = " ",
            row.names = TRUE, col.names = TRUE)

In [146]:
rownames(logfcvals) <- sub('\\.[0-9]*$', '', rownames(logfcvals))
head(logfcvals)

log2 fold change (MLE): condition B vs A 
Wald test p-value: condition B vs A 
DataFrame with 6 rows and 6 columns
                   baseMean log2FoldChange     lfcSE      stat      pvalue
                  <numeric>      <numeric> <numeric> <numeric>   <numeric>
ENSG00000000005   113.07162      -7.376670  1.527138  -4.83039 1.36267e-06
ENSG00000001561   412.90442      -0.705891  0.429949  -1.64180 1.00631e-01
ENSG00000004799  9057.73913      -2.522021  1.010229  -2.49648 1.25431e-02
ENSG00000004848     3.37816       2.594541  1.042167   2.48956 1.27900e-02
ENSG00000005022 12944.44290       0.914033  0.277609   3.29252 9.92944e-04
ENSG00000005073   305.59313       0.787575  0.760817   1.03517 3.00589e-01
                       padj
                  <numeric>
ENSG00000000005 1.81107e-05
ENSG00000001561 2.39523e-01
ENSG00000004799 4.69391e-02
ENSG00000004848 4.76250e-02
ENSG00000005022 5.75950e-03
ENSG00000005073 5.14246e-01

In [147]:
human <- useMart("ensembl", dataset="hsapiens_gene_ensembl")

In [148]:
logfc_ensembl <- getLDS(attributes=c("ensembl_gene_id"), filters="ensembl_gene_id",
                          values=rownames(logfcvals),
                          mart=human,attributesL=c("hgnc_symbol","ensembl_gene_id"),
                          martL=human)

In [152]:
dim(logfc_ensembl)
head(logfc_ensembl)

[1] 30016     3

,Gene.stable.ID,HGNC.symbol,Gene.stable.ID.1
,<chr>,<chr>,<chr>
1,ENSG00000210049,MT-TF,ENSG00000210049
2,ENSG00000211459,MT-RNR1,ENSG00000211459
3,ENSG00000210077,MT-TV,ENSG00000210077
4,ENSG00000210082,MT-RNR2,ENSG00000210082
5,ENSG00000209082,MT-TL1,ENSG00000209082
6,ENSG00000198888,MT-ND1,ENSG00000198888


In [151]:
length(rownames(logfcvals) %in% logfc_ensembl$Gene.stable.ID)

[1] 31919

In [135]:
listAttributes(human)

name,description,page
<chr>,<chr>,<chr>
ensembl_gene_id,Gene stable ID,feature_page
ensembl_gene_id_version,Gene stable ID version,feature_page
ensembl_transcript_id,Transcript stable ID,feature_page
ensembl_transcript_id_version,Transcript stable ID version,feature_page
ensembl_peptide_id,Protein stable ID,feature_page
ensembl_peptide_id_version,Protein stable ID version,feature_page
ensembl_exon_id,Exon stable ID,feature_page
description,Gene description,feature_page
chromosome_name,Chromosome/scaffold name,feature_page


In [138]:
rownames(logfcvals)

[1] "ENSG00000000005.5"  "ENSG00000001561.6"  "ENSG00000004799.7" 
    [4] "ENSG00000004848.6"  "ENSG00000005022.5"  "ENSG00000005073.5" 
    [7] "ENSG00000005513.9"  "ENSG00000006059.3"  "ENSG00000006116.3" 
   [10] "ENSG00000006210.6"  "ENSG00000006831.9"  "ENSG00000007520.3" 
   [13] "ENSG00000007968.6"  "ENSG00000008018.8"  "ENSG00000008197.4" 
   [16] "ENSG00000008438.4"  "ENSG00000013293.5"  "ENSG00000013588.5" 
   [19] "ENSG00000014123.9"  "ENSG00000014164.6"  "ENSG00000019995.6" 
   [22] "ENSG00000023318.7"  "ENSG00000023608.4"  "ENSG00000025772.7" 
   [25] "ENSG00000027644.4"  "ENSG00000031691.6"  "ENSG00000035687.9" 
   [28] "ENSG00000036473.6"  "ENSG00000037042.8"  "ENSG00000039139.9" 
   [31] "ENSG00000044012.3"  "ENSG00000044574.7"  "ENSG00000046774.9" 
   [34] "ENSG00000047597.5"  "ENSG00000047662.4"  "ENSG00000048544.5" 
   [37] "ENSG00000050344.8"  "ENSG00000052850.5"  "ENSG00000053328.8" 
   [40] "ENSG00000053372.4"  "ENSG00000054803.3"  "ENSG00000055813.5" 
   [43] "ENSG00000056050.6"  "ENSG00000057468.6"  "ENSG00000060303.5" 
   [46] "ENSG00000060558.3"  "ENSG00000061455.10" "ENSG00000063515.2" 
   [49] "ENSG00000064195.7"  "ENSG00000064218.4"  "ENSG00000065308.4" 
   [52] "ENSG00000065491.8"  "ENSG00000065518.7"  "ENSG00000066422.4" 
   [55] "ENSG00000066557.5"  "ENSG00000067533.5"  "ENSG00000067900.6" 
   [58] "ENSG00000068697.6"  "ENSG00000068985.4"  "ENSG00000069275.12"
   [61] "ENSG00000069482.6"  "ENSG00000069509.5"  "ENSG00000069696.6" 
   [64] "ENSG00000070031.3"  "ENSG00000070193.4"  "ENSG00000071677.1" 
   [67] "ENSG00000072315.3"  "ENSG00000073598.5"  "ENSG00000073734.8" 
   [70] "ENSG00000073754.5"  "ENSG00000073861.2"  "ENSG00000073905.8" 
   [73] "ENSG00000074695.5"  "ENSG00000074771.3"  "ENSG00000075290.7" 
   [76] "ENSG00000075461.5"  "ENSG00000075643.5"  "ENSG00000075886.10"
   [79] "ENSG00000076003.4"  "ENSG00000076356.6"  "ENSG00000076641.4" 
   [82] "ENSG00000077498.8"  "ENSG00000077943.7"  "ENSG00000078401.6" 
   [85] "ENSG00000078579.8"  "ENSG00000078898.6"  "ENSG00000080573.6" 
   [88] "ENSG00000080608.9"  "ENSG00000081041.8"  "ENSG00000081148.11"
   [91] "ENSG00000082516.8"  "ENSG00000083622.8"  "ENSG00000083635.7" 
   [94] "ENSG00000083817.8"  "ENSG00000084092.6"  "ENSG00000086506.2" 
   [97] "ENSG00000086544.2"  "ENSG00000086548.8"  "ENSG00000086967.9" 
  [100] "ENSG00000087074.7"  "ENSG00000087338.4"  "ENSG00000088320.3" 
  [103] "ENSG00000088538.12" "ENSG00000088782.4"  "ENSG00000088882.7" 
  [106] "ENSG00000089116.3"  "ENSG00000089163.4"  "ENSG00000089220.4" 
  [109] "ENSG00000089248.6"  "ENSG00000090776.5"  "ENSG00000090971.4" 
  [112] "ENSG00000091010.4"  "ENSG00000091317.7"  "ENSG00000091482.5" 
  [115] "ENSG00000091483.6"  "ENSG00000091542.8"  "ENSG00000091664.7" 
  [118] "ENSG00000092054.12" "ENSG00000092067.5"  "ENSG00000094661.2" 
  [121] "ENSG00000094796.4"  "ENSG00000095015.5"  "ENSG00000095380.10"
  [124] "ENSG00000095464.9"  "ENSG00000095587.8"  "ENSG00000096092.5" 
  [127] "ENSG00000096401.7"  "ENSG00000097096.8"  "ENSG00000099194.5" 
  [130] "ENSG00000099399.5"  "ENSG00000099769.5"  "ENSG00000099822.2" 
  [133] "ENSG00000099974.7"  "ENSG00000099985.3"  "ENSG00000099994.10"
  [136] "ENSG00000100078.3"  "ENSG00000100122.5"  "ENSG00000100234.11"
  [139] "ENSG00000100249.4"  "ENSG00000100276.9"  "ENSG00000100285.9" 
  [142] "ENSG00000100290.2"  "ENSG00000100302.6"  "ENSG00000100304.12"
  [145] "ENSG00000100314.3"  "ENSG00000100387.8"  "ENSG00000100393.9" 
  [148] "ENSG00000100418.7"  "ENSG00000100426.6"  "ENSG00000100448.3" 
  [151] "ENSG00000100580.7"  "ENSG00000100583.4"  "ENSG00000100625.8" 
  [154] "ENSG00000100647.7"  "ENSG00000100652.4"  "ENSG00000100985.7" 
  [157] "ENSG00000101000.4"  "ENSG00000101074.3"  "ENSG00000101162.3" 
  [160] "ENSG00000101188.4"  "ENSG00000101200.5"  "ENSG00000101230.5" 
  [163] "ENSG00000101278.6"  "ENSG00000101323.4"  "ENSG00000101367.8" 
  [166] "ENSG00000101400.5"  "ENSG00000101405.3"  "ENSG00000101421.3" 
  [169] "ENSG00000101435.4" 